In [9]:
!pip install --upgrade pip setuptools
!pip install numpy==1.19.1
!pip install -r requirements.txt

  Using cached numpy-1.19.1.zip (7.3 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  error: subprocess-exited-with-error
  
  × Preparing metadata (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (pyproject.toml) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.
ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'


In [10]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
!pip install docopt

In [ ]:
output_dir = '/content/drive/My Drive/output'
import os
os.makedirs(output_dir, exist_ok=True)

In [ ]:
!python hover_net-master/run_infer.py --gpu=0 --nr_types=6 --type_info_path=hover_net-master/type_info.json --model_path=hover_net-master/models/hovernet_fast_pannuke_type_tf2pytorch.tar --model_mode=fast tile --input_dir=hover_net-master/HE_old_40X --output_dir='/content/drive/My Drive/output' --save_qupath

|2024-07-11|20:59:23.870| [INFO] .... Detect #GPUS: 1
Process Patches: 136it [08:44,  3.86s/it]
|2024-07-11|21:09:04.073| [INFO] ........................ Done Assembling D-3
|2024-07-11|21:10:12.160| [INFO] ........................ Done Assembling L-2
|2024-07-11|21:10:35.629| [INFO] ........................ Done Assembling B-8
|2024-07-11|21:10:43.933| [INFO] ........................ Done Assembling P-1
|2024-07-11|21:10:59.896| [INFO] ........................ Done Assembling L-8


In [12]:
!pip install geojson

In [ ]:
import sys
sys.argv = ['script_name.py', '/content/B-8.json', '/content/B-8.geojson']

import argparse
import gzip
import json
from pathlib import Path
from shapely.geometry import Polygon

# Type mapping dictionary for HoverNet output
type_mapping = {
    0: "nolabel",
    1: "neoplastic",
    2: "inflammatory",
    3: "connective",
    4: "necrosis",
    5: "non-neoplastic",
}

def hovernet_to_feature_dict(instance_id: int, d: dict):
    """Convert HoverNet instance dictionary to GeoJSON feature dictionary."""
    poly = Polygon(d["contour"])
    coordinates = [list(coord) for coord in poly.exterior.coords]  # Convert to list for JSON serialization
    return {
        "type": "Feature",
        "geometry": {
            "type": "Polygon",
            "coordinates": [coordinates],
        },
        "properties": {
            "classification": type_mapping[d["type"]],
            "instance_id": int(instance_id),
            "type_str": type_mapping[d["type"]],
            "type_int": d["type"],
            "type_prob": d["type_prob"],
        },
    }

def hovernet_to_geojson(d: dict):
    """Convert HoverNet output dictionary to GeoJSON format."""
    assert "nuc" in d.keys(), "expected 'nuc' key"
    assert "mag" in d.keys(), "expected 'mag' key"
    #assert d["mag"] == 40, "this script was designed for mag=40x"
    features = [hovernet_to_feature_dict(ii, dd) for ii, dd in d["nuc"].items()]
    return {
        "type": "FeatureCollection",
        "features": features,
    }

if __name__ == "__main__":
    parser = argparse.ArgumentParser(description=__doc__)
    parser.add_argument("input", help="JSON file with HoverNet predictions.")
    parser.add_argument("output", help="Output GeoJSON file")
    args = parser.parse_args()

    input_path = Path(args.input)
    output_path = Path(args.output)

    if not input_path.exists():
        raise FileNotFoundError(f"Input file not found: {args.input}")
    if output_path.exists():
        raise FileExistsError(f"Output file exists: {args.output}")

    print("Reading HoverNet polygons...")
    open_fn = gzip.open if args.input.endswith(".gz") else open
    with open_fn(args.input) as f:
        d = json.load(f)

    print("Converting to GeoJSON...")
    geojson = hovernet_to_geojson(d)

    print(f"Writing output to {args.output}")
    open_fn = gzip.open if args.output.endswith(".gz") else open
    with open_fn(args.output, "wt") as f:
        json.dump(geojson, f)

    print("Done.")


Reading HoverNet polygons...
Converting to GeoJSON...
Writing output to /content/B-8.geojson
Done.


In [ ]:
import sys
sys.argv = ['script_name.py', '/content/D-3.json', '/content/D-3.geojson']

import argparse
import gzip
import json
from pathlib import Path
from shapely.geometry import Polygon

# Type mapping dictionary for HoverNet output
type_mapping = {
    0: "nolabel",
    1: "neoplastic",
    2: "inflammatory",
    3: "connective",
    4: "necrosis",
    5: "non-neoplastic",
}

def hovernet_to_feature_dict(instance_id: int, d: dict):
    """Convert HoverNet instance dictionary to GeoJSON feature dictionary."""
    poly = Polygon(d["contour"])
    coordinates = [list(coord) for coord in poly.exterior.coords]  # Convert to list for JSON serialization
    return {
        "type": "Feature",
        "geometry": {
            "type": "Polygon",
            "coordinates": [coordinates],
        },
        "properties": {
            "classification": type_mapping[d["type"]],
            "instance_id": int(instance_id),
            "type_str": type_mapping[d["type"]],
            "type_int": d["type"],
            "type_prob": d["type_prob"],
        },
    }

def hovernet_to_geojson(d: dict):
    """Convert HoverNet output dictionary to GeoJSON format."""
    assert "nuc" in d.keys(), "expected 'nuc' key"
    assert "mag" in d.keys(), "expected 'mag' key"
    #assert d["mag"] == 40, "this script was designed for mag=40x"
    features = [hovernet_to_feature_dict(ii, dd) for ii, dd in d["nuc"].items()]
    return {
        "type": "FeatureCollection",
        "features": features,
    }

if __name__ == "__main__":
    parser = argparse.ArgumentParser(description=__doc__)
    parser.add_argument("input", help="JSON file with HoverNet predictions.")
    parser.add_argument("output", help="Output GeoJSON file")
    args = parser.parse_args()

    input_path = Path(args.input)
    output_path = Path(args.output)

    if not input_path.exists():
        raise FileNotFoundError(f"Input file not found: {args.input}")
    if output_path.exists():
        raise FileExistsError(f"Output file exists: {args.output}")

    print("Reading HoverNet polygons...")
    open_fn = gzip.open if args.input.endswith(".gz") else open
    with open_fn(args.input) as f:
        d = json.load(f)

    print("Converting to GeoJSON...")
    geojson = hovernet_to_geojson(d)

    print(f"Writing output to {args.output}")
    open_fn = gzip.open if args.output.endswith(".gz") else open
    with open_fn(args.output, "wt") as f:
        json.dump(geojson, f)

    print("Done.")


Reading HoverNet polygons...
Converting to GeoJSON...
Writing output to /content/D-3.geojson
Done.


In [ ]:
import sys
sys.argv = ['script_name.py', '/content/L-2.json', '/content/L-2.geojson']

import argparse
import gzip
import json
from pathlib import Path
from shapely.geometry import Polygon

# Type mapping dictionary for HoverNet output
type_mapping = {
    0: "nolabel",
    1: "neoplastic",
    2: "inflammatory",
    3: "connective",
    4: "necrosis",
    5: "non-neoplastic",
}

def hovernet_to_feature_dict(instance_id: int, d: dict):
    """Convert HoverNet instance dictionary to GeoJSON feature dictionary."""
    poly = Polygon(d["contour"])
    coordinates = [list(coord) for coord in poly.exterior.coords]  # Convert to list for JSON serialization
    return {
        "type": "Feature",
        "geometry": {
            "type": "Polygon",
            "coordinates": [coordinates],
        },
        "properties": {
            "classification": type_mapping[d["type"]],
            "instance_id": int(instance_id),
            "type_str": type_mapping[d["type"]],
            "type_int": d["type"],
            "type_prob": d["type_prob"],
        },
    }

def hovernet_to_geojson(d: dict):
    """Convert HoverNet output dictionary to GeoJSON format."""
    assert "nuc" in d.keys(), "expected 'nuc' key"
    assert "mag" in d.keys(), "expected 'mag' key"
    #assert d["mag"] == 40, "this script was designed for mag=40x"
    features = [hovernet_to_feature_dict(ii, dd) for ii, dd in d["nuc"].items()]
    return {
        "type": "FeatureCollection",
        "features": features,
    }

if __name__ == "__main__":
    parser = argparse.ArgumentParser(description=__doc__)
    parser.add_argument("input", help="JSON file with HoverNet predictions.")
    parser.add_argument("output", help="Output GeoJSON file")
    args = parser.parse_args()

    input_path = Path(args.input)
    output_path = Path(args.output)

    if not input_path.exists():
        raise FileNotFoundError(f"Input file not found: {args.input}")
    if output_path.exists():
        raise FileExistsError(f"Output file exists: {args.output}")

    print("Reading HoverNet polygons...")
    open_fn = gzip.open if args.input.endswith(".gz") else open
    with open_fn(args.input) as f:
        d = json.load(f)

    print("Converting to GeoJSON...")
    geojson = hovernet_to_geojson(d)

    print(f"Writing output to {args.output}")
    open_fn = gzip.open if args.output.endswith(".gz") else open
    with open_fn(args.output, "wt") as f:
        json.dump(geojson, f)

    print("Done.")


Reading HoverNet polygons...
Converting to GeoJSON...
Writing output to /content/L-2.geojson
Done.


In [ ]:
import sys
sys.argv = ['script_name.py', '/content/L-8.json', '/content/L-8.geojson']

import argparse
import gzip
import json
from pathlib import Path
from shapely.geometry import Polygon

# Type mapping dictionary for HoverNet output
type_mapping = {
    0: "nolabel",
    1: "neoplastic",
    2: "inflammatory",
    3: "connective",
    4: "necrosis",
    5: "non-neoplastic",
}

def hovernet_to_feature_dict(instance_id: int, d: dict):
    """Convert HoverNet instance dictionary to GeoJSON feature dictionary."""
    poly = Polygon(d["contour"])
    coordinates = [list(coord) for coord in poly.exterior.coords]  # Convert to list for JSON serialization
    return {
        "type": "Feature",
        "geometry": {
            "type": "Polygon",
            "coordinates": [coordinates],
        },
        "properties": {
            "classification": type_mapping[d["type"]],
            "instance_id": int(instance_id),
            "type_str": type_mapping[d["type"]],
            "type_int": d["type"],
            "type_prob": d["type_prob"],
        },
    }

def hovernet_to_geojson(d: dict):
    """Convert HoverNet output dictionary to GeoJSON format."""
    assert "nuc" in d.keys(), "expected 'nuc' key"
    assert "mag" in d.keys(), "expected 'mag' key"
    #assert d["mag"] == 40, "this script was designed for mag=40x"
    features = [hovernet_to_feature_dict(ii, dd) for ii, dd in d["nuc"].items()]
    return {
        "type": "FeatureCollection",
        "features": features,
    }

if __name__ == "__main__":
    parser = argparse.ArgumentParser(description=__doc__)
    parser.add_argument("input", help="JSON file with HoverNet predictions.")
    parser.add_argument("output", help="Output GeoJSON file")
    args = parser.parse_args()

    input_path = Path(args.input)
    output_path = Path(args.output)

    if not input_path.exists():
        raise FileNotFoundError(f"Input file not found: {args.input}")
    if output_path.exists():
        raise FileExistsError(f"Output file exists: {args.output}")

    print("Reading HoverNet polygons...")
    open_fn = gzip.open if args.input.endswith(".gz") else open
    with open_fn(args.input) as f:
        d = json.load(f)

    print("Converting to GeoJSON...")
    geojson = hovernet_to_geojson(d)

    print(f"Writing output to {args.output}")
    open_fn = gzip.open if args.output.endswith(".gz") else open
    with open_fn(args.output, "wt") as f:
        json.dump(geojson, f)

    print("Done.")


Reading HoverNet polygons...
Converting to GeoJSON...
Writing output to /content/L-8.geojson
Done.


In [ ]:
import sys
sys.argv = ['script_name.py', '/content/P-1.json', '/content/P-1.geojson']

import argparse
import gzip
import json
from pathlib import Path
from shapely.geometry import Polygon

# Type mapping dictionary for HoverNet output
type_mapping = {
    0: "nolabel",
    1: "neoplastic",
    2: "inflammatory",
    3: "connective",
    4: "necrosis",
    5: "non-neoplastic",
}

def hovernet_to_feature_dict(instance_id: int, d: dict):
    """Convert HoverNet instance dictionary to GeoJSON feature dictionary."""
    poly = Polygon(d["contour"])
    coordinates = [list(coord) for coord in poly.exterior.coords]  # Convert to list for JSON serialization
    return {
        "type": "Feature",
        "geometry": {
            "type": "Polygon",
            "coordinates": [coordinates],
        },
        "properties": {
            "classification": type_mapping[d["type"]],
            "instance_id": int(instance_id),
            "type_str": type_mapping[d["type"]],
            "type_int": d["type"],
            "type_prob": d["type_prob"],
        },
    }

def hovernet_to_geojson(d: dict):
    """Convert HoverNet output dictionary to GeoJSON format."""
    assert "nuc" in d.keys(), "expected 'nuc' key"
    assert "mag" in d.keys(), "expected 'mag' key"
    #assert d["mag"] == 40, "this script was designed for mag=40x"
    features = [hovernet_to_feature_dict(ii, dd) for ii, dd in d["nuc"].items()]
    return {
        "type": "FeatureCollection",
        "features": features,
    }

if __name__ == "__main__":
    parser = argparse.ArgumentParser(description=__doc__)
    parser.add_argument("input", help="JSON file with HoverNet predictions.")
    parser.add_argument("output", help="Output GeoJSON file")
    args = parser.parse_args()

    input_path = Path(args.input)
    output_path = Path(args.output)

    if not input_path.exists():
        raise FileNotFoundError(f"Input file not found: {args.input}")
    if output_path.exists():
        raise FileExistsError(f"Output file exists: {args.output}")

    print("Reading HoverNet polygons...")
    open_fn = gzip.open if args.input.endswith(".gz") else open
    with open_fn(args.input) as f:
        d = json.load(f)

    print("Converting to GeoJSON...")
    geojson = hovernet_to_geojson(d)

    print(f"Writing output to {args.output}")
    open_fn = gzip.open if args.output.endswith(".gz") else open
    with open_fn(args.output, "wt") as f:
        json.dump(geojson, f)

    print("Done.")


Reading HoverNet polygons...
Converting to GeoJSON...
Writing output to /content/P-1.geojson
Done.


In [ ]:
from google.colab import files
files.download('B-8.geojson')
files.download('D-3.geojson')
files.download('L-2.geojson')
files.download('L-8.geojson')
files.download('P-1.geojson')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import json
import scipy.io as sio
import os

# Define the list of GeoJSON files
geojson_files = ['B-8.geojson', 'L-8.geojson', 'P-1.geojson', 'D-3.geojson', 'L-2.geojson']

# Function to convert GeoJSON to MAT
def geojson_to_mat(geojson_file):
    # Create the corresponding .mat file name
    mat_file = geojson_file.replace('.geojson', '.mat')

    # Read the GeoJSON file
    with open(geojson_file, 'r') as f:
        geojson_data = json.load(f)

    # Initialize lists to store data
    coordinates = []
    object_types = []
    classifications = []

    # Loop through each feature to extract data
    for feature in geojson_data['features']:
        # Extract coordinates
        coords = feature['geometry']['coordinates']
        coordinates.append(coords)

        # Extract object type
        obj_type = feature['properties']['objectType']
        object_types.append(obj_type)

        # Extract classification name and color
        class_name = feature['properties']['classification']['name']
        class_color = feature['properties']['classification']['color']
        classifications.append({'name': class_name, 'color': class_color})

    # Save the data into a .mat file
    sio.savemat(mat_file, {'coordinates': coordinates, 'object_types': object_types, 'classifications': classifications})

# Process each GeoJSON file
for geojson_file in geojson_files:
    geojson_to_mat(geojson_file)

from google.colab import files
files.download('B-8.geojson')
files.download('D-3.geojson')
files.download('L-2.geojson')
files.download('L-8.geojson')
files.download('P-1.geojson')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
label_path = '/content/hovernet/data/labels/B-8.mat'


In [ ]:
!unzip '/content/drive/MyDrive/hovernet (1).zip'

Archive:  /content/drive/MyDrive/hovernet (1).zip
  inflating: hovernet/.gitignore     
   creating: hovernet/.vscode/
  inflating: hovernet/.vscode/launch.json  
 extracting: hovernet/__init__.py    
   creating: hovernet/__pycache__/
  inflating: hovernet/__pycache__/config.cpython-311.pyc  
  inflating: hovernet/__pycache__/convert_format.cpython-311.pyc  
  inflating: hovernet/__pycache__/dataset.cpython-311.pyc  
  inflating: hovernet/CHANGELOG.md   
   creating: hovernet/checkpoints/
  inflating: hovernet/compute_stats.py  
  inflating: hovernet/config.py      
  inflating: hovernet/convert_chkpt_tf2pytorch.py  
  inflating: hovernet/convert_format.py  
   creating: hovernet/data/
   creating: hovernet/data/images/
  inflating: hovernet/data/images/B-8.tiff  
  inflating: hovernet/data/images/D-3.tiff  
  inflating: hovernet/data/images/L-2.tiff  
  inflating: hovernet/data/images/L-8.tiff  
  inflating: hovernet/data/images/P-1.tiff  
   creating: hovernet/data/labels/
  inflati

In [ ]:
!pip install tensorboardX
import tensorboardX
!pip install tqdm

In [ ]:
import json
import numpy as np
import cv2
import scipy.io as sio

def convert_geojson_to_mat(geojson_file, mat_file, image_size=(5000, 5000)):
    # Load the GeoJSON data
    with open(geojson_file, 'r') as f:
        geojson_data = json.load(f)

    # Initialize lists to store extracted data
    coordinates_list = []
    color_list = []

    # Extract data from each feature
    for feature in geojson_data['features']:
        coordinates = feature['geometry']['coordinates'][0]  # Assuming a single polygon per feature
        color = feature['properties']['classification']['color']

        coordinates_list.append(np.array(coordinates))
        color_list.append(color)

    # Create maps
    inst_map = np.zeros(image_size, np.int32)
    type_map = np.zeros(image_size, np.int32)
    color_map = np.zeros(image_size + (3,), np.int32)  # For RGB color

    # Draw each polygon on the maps
    for idx, coordinates in enumerate(coordinates_list):
        # Convert coordinates to integer and draw on the maps
        polygon = coordinates.astype(np.int32)
        cv2.fillPoly(inst_map, [polygon], idx + 1)
        cv2.fillPoly(type_map, [polygon], idx + 1)
        cv2.fillPoly(color_map, [polygon], color_list[idx])

    # Prepare the data dictionary
    data_dict = {
        'inst_map': inst_map,
        'type_map': type_map,
        'color_map': color_map
    }

    # Save to .mat file
    sio.savemat(mat_file, data_dict)

files_to_convert = ['B-8', 'D-3', 'L-2', 'L-8', 'P-1']
for file_name in files_to_convert:
    convert_geojson_to_mat(f'{file_name}.geojson', f'{file_name}.mat')

from google.colab import files
files.download('B-8.mat')
files.download('D-3.mat')
files.download('L-2.mat')
files.download('L-8.mat')
files.download('P-1.mat')


FileNotFoundError: [Errno 2] No such file or directory: 'B-8.geojson'

In [ ]:
import os

img_dir = "/content/hovernet/data/images"
base_name = "B-8"
img_ext = ".tiff"
img_path = os.path.join(img_dir, base_name + img_ext)
print(img_path)

# Check if the file exists
print(os.path.exists(img_path))  # Should print True if the file exists

/content/hovernet/data/images/B-8.tiff
True


In [ ]:
import scipy.io as sio
import os
import glob

# Define the directory and file extension
ann_dir = "/content/hovernet/data/labels"
ann_ext = ".mat"

# Construct the file pattern
file_pattern = os.path.join(ann_dir, f"*{ann_ext}")

# Get the list of files matching the pattern
file_list = glob.glob(file_pattern)
file_list.sort()  # Ensure the same ordering across platforms

# Print the list of matched files
print("Matched Files:", file_list)

# Function to load .mat file and handle KeyError
def load_ann(path):
    try:
        mat_data = sio.loadmat(path)
        # Print keys for debugging
        print(f"Keys in {path}:", mat_data.keys())

        # Attempt to load 'inst_map'
        if 'inst_map' in mat_data:
            ann_inst = mat_data['inst_map']
            print(f"Loaded 'inst_map' from {path}")
            return ann_inst
        else:
            print(f"'inst_map' key not found in {path}")
            return None
    except KeyError as e:
        print(f"KeyError: {e} in file {path}")
        return None

# Load each annotation file
for ann_file in file_list:
    ann_inst = load_ann(ann_file)
    # Perform further processing with ann_inst if needed


Matched Files: ['/content/hovernet/data/labels/B-8.mat', '/content/hovernet/data/labels/D-3.mat', '/content/hovernet/data/labels/L-2.mat', '/content/hovernet/data/labels/L-8.mat', '/content/hovernet/data/labels/P-1.mat']
Keys in /content/hovernet/data/labels/B-8.mat: dict_keys(['__header__', '__version__', '__globals__', 'inst_map', 'type_map', 'color_map'])
Loaded 'inst_map' from /content/hovernet/data/labels/B-8.mat
Keys in /content/hovernet/data/labels/D-3.mat: dict_keys(['__header__', '__version__', '__globals__', 'inst_map', 'type_map', 'color_map'])
Loaded 'inst_map' from /content/hovernet/data/labels/D-3.mat
Keys in /content/hovernet/data/labels/L-2.mat: dict_keys(['__header__', '__version__', '__globals__', 'inst_map', 'type_map', 'color_map'])
Loaded 'inst_map' from /content/hovernet/data/labels/L-2.mat
Keys in /content/hovernet/data/labels/L-8.mat: dict_keys(['__header__', '__version__', '__globals__', 'inst_map', 'type_map', 'color_map'])
Loaded 'inst_map' from /content/hove

In [ ]:
!python hovernet/extract_patches.py

  0% 0/5 [00:00<?, ?it/s]Original img shape: (4972, 4972, 3)
Original ann shape: (5000, 5000, 2)
Adjusted img shape: (5000, 5000, 3)
Adjusted ann shape: (5000, 5000, 2)

  0% 0/961 [00:00<?, ?it/s]
  0% 2/961 [00:00<00:57, 16.71it/s]
  1% 5/961 [00:00<00:49, 19.17it/s]
  1% 8/961 [00:00<00:47, 20.22it/s]
  1% 11/961 [00:00<00:50, 18.79it/s]
  1% 13/961 [00:00<00:50, 18.84it/s]
  2% 15/961 [00:00<00:50, 18.84it/s]
  2% 18/961 [00:00<00:48, 19.33it/s]
  2% 21/961 [00:01<00:47, 19.84it/s]
  2% 23/961 [00:01<00:48, 19.25it/s]
  3% 25/961 [00:01<00:52, 17.69it/s]
  3% 27/961 [00:01<00:53, 17.54it/s]
  3% 29/961 [00:01<00:52, 17.78it/s]
  3% 32/961 [00:01<00:48, 19.19it/s]
  4% 34/961 [00:01<00:48, 19.12it/s]
  4% 36/961 [00:01<00:47, 19.28it/s]
  4% 38/961 [00:02<00:48, 18.96it/s]
  4% 40/961 [00:02<00:48, 19.04it/s]
  4% 42/961 [00:02<00:49, 18.71it/s]
  5% 44/961 [00:02<00:49, 18.61it/s]
  5% 46/961 [00:02<00:49, 18.30it/s]
  5% 48/961 [00:02<00:51, 17.60it/s]
  5% 50/961 [00:02<00:51, 17

In [ ]:
!curl https://pyenv.run | bash
!pyenv install -v 3.8.1


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   270  100   270    0     0   1030      0 --:--:-- --:--:-- --:--:--  1034
Cloning into '/root/.pyenv'...
remote: Enumerating objects: 1269, done.
remote: Counting objects: 100% (1269/1269), done.
remote: Compressing objects: 100% (701/701), done.
remote: Total 1269 (delta 748), reused 735 (delta 435), pack-reused 0
Receiving objects: 100% (1269/1269), 627.86 KiB | 6.28 MiB/s, done.
Resolving deltas: 100% (748/748), done.
Cloning into '/root/.pyenv/plugins/pyenv-doctor'...
remote: Enumerating objects: 11, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 11 (delta 1), reused 5 (delta 0), pack-reused 0
Receiving objects: 100% (11/11), 38.72 KiB | 2.98 MiB/s, done.
Resolving deltas: 100% (1/1), done.
Cloning into '/root/.pyenv/plugins/pyenv-update'...
remote: Enumera

In [ ]:
!python --version
!pip uninstall numpy
!pip install numpy==1.19.1
#!pip install numpy==1.19.1

Python 3.10.12
  Using cached numpy-1.19.1.zip (7.3 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  error: subprocess-exited-with-error
  
  × Preparing metadata (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (pyproject.toml) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [ ]:
!python hovernet/run_train.py --gpu=0 #--view='train'
#!python hover_net-master/run_infer.py --gpu=0 --nr_types=6 --type_info_path=hover_net-master/type_info.json --model_path=hover_net-master/models/hovernet_fast_pannuke_type_tf2pytorch.tar --model_mode=fast tile --input_dir=hover_net-master/HE_old_40X --output_dir='/content/drive/My Drive/output' --save_qupath

Detect #GPUS: 1
Using manual seed: 10
Dataset train: 4805
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 8, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
Dataset valid: 4805
Model `net` pretrained path: /content/hovernet/ImageNet-ResNet50-Preact_pytorch.tar
Missing Variables: 
 ['conv_bot.weight', 'decoder.tp.u3.conva.weight', 'decoder.tp.u3.dense.units.0.preact_bna/bn.weight', 'decoder.tp.u3.dense.units.0.preact_bna/bn.bias', 'decoder.tp.u3.dense.units.0.preact_bna/bn.running_mean', 'decoder.tp.u3.dense.units.0.preact_bna/bn.running_var', 'decoder.tp.u3.dense.units.0.conv1.weight', 'decoder.tp.u3.dense.units.0.c

In [ ]:
import numpy
numpy.version.version

'1.26.4'

In [ ]:
emphasis on:
- self.dataset_name = "aml" in comfig.py
- fix tensor hot fixing
- print batch_data -> in run_desc see whats inside
